<a href="https://colab.research.google.com/github/olgasherbiena/KPI/blob/main/%D0%90%D0%9A_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub


In [2]:
import pandas as pd
import kagglehub
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

csv_path  = kagglehub.dataset_download("shantanudhakadd/email-spam-detection-dataset-classification")
csv_file = os.path.join(csv_path, 'spam.csv')
df = pd.read_csv(csv_file, encoding='latin1')

100%|██████████| 211k/211k [00:00<00:00, 6.74MB/s]

Extracting files...


In [3]:
df = df[['v1', 'v2']].copy()
df.columns = ['label', 'text']
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

print(df.head())
print(df['label'].value_counts())

   label                                               text
0      0  Go until jurong point, crazy.. Available only ...
1      0                      Ok lar... Joking wif u oni...
2      1  Free entry in 2 a wkly comp to win FA Cup fina...
3      0  U dun say so early hor... U c already then say...
4      0  Nah I don't think he goes to usf, he lives aro...
label
0    4825
1     747
Name: count, dtype: int64


In [4]:
import numpy as np

class LSH:
    def __init__(self, num_buckets, num_hash_functions, input_dim):
        self.num_buckets = num_buckets
        self.num_hash_functions = num_hash_functions
        self.input_dim = input_dim
        self.hash_functions = [self._generate_hash_function() for _ in range(num_hash_functions)]
        self.buckets = {i: [] for i in range(num_buckets)}

    def _generate_hash_function(self):
        return np.random.randn(self.input_dim)

    def hash(self, vector):
        hashes = []
        for hash_function in self.hash_functions:
            hash_value = np.dot(vector, hash_function)
            hash_value = hash_value / np.linalg.norm(hash_function)
            hash_value = hash_value.astype(int)
            hashes.append(hash_value)
        return hashes

    def add_vector(self, vector):
        hashes = self.hash(vector)
        for i, hash_value in enumerate(hashes):
            self.buckets[hash_value % self.num_buckets].append((i, vector))

    def query(self, query_vector):
        query_hashes = self.hash(query_vector)
        similar_items = []
        for hash_value in query_hashes:
            bucket = self.buckets[hash_value % self.num_buckets]
            for item in bucket:
                if np.array_equal(item[1], query_vector):
                    continue
                if not any(np.array_equal(item[1], sim[1]) for sim in similar_items):
                    similar_items.append(item)
        return similar_items


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

vectorizer = CountVectorizer()

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['text'])

svd = TruncatedSVD(n_components=50)
X_dense = svd.fit_transform(X_tfidf)

In [6]:
print(X_dense.shape)
#X_small = X_dense
X_small = X_dense[:500]

(5572, 50)


In [ ]:
lsh = LSH(num_buckets=10, num_hash_functions=5, input_dim=X_small.shape[1])
for vec in X_small:
    lsh.add_vector(vec)
query_vec = X_small[0]
similar_items = lsh.query(query_vec)
print("Схожі об'єкти:", len(similar_items))

Схожі об'єкти: 489


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

similar_items_sorted = sorted(
    similar_items,
    key=lambda x: cosine_similarity(query_vec.reshape(1, -1), x[1].reshape(1, -1))[0][0],
    reverse=True  # від більшого до меншого
)

In [ ]:
for idx, (hash_id, vec) in enumerate(similar_items_sorted[:10]):
    cos_sim = cosine_similarity(query_vec.reshape(1, -1), vec.reshape(1, -1))[0][0]
    print(f"#{idx+1} | Cosine similarity: {cos_sim:.2f}")

#1 | Cosine similarity: 0.33
#2 | Cosine similarity: 0.31
#3 | Cosine similarity: 0.29
#4 | Cosine similarity: 0.27
#5 | Cosine similarity: 0.26
#6 | Cosine similarity: 0.26
#7 | Cosine similarity: 0.26
#8 | Cosine similarity: 0.25
#9 | Cosine similarity: 0.25
#10 | Cosine similarity: 0.25


In [ ]:
print(f"Запитний лист:\n{df['text']}\nLabel: {'Spam' if df['label'].iloc[0]==1 else 'Ham'}\n")

# Виведемо перші 10 схожих листів
for idx, (i, vec) in enumerate(similar_items_sorted[:10]):
    similarity = np.dot(query_vec, vec) / (np.linalg.norm(query_vec) * np.linalg.norm(vec))
    print(f"#{idx+1} | Label: {'Spam' if df['label'].iloc[i]==1 else 'Ham'} | Cosine similarity: {similarity:.2f}")
    print(f"Text: {df['text']}...")


Запитний лист:
0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: text, Length: 5572, dtype: object
Label: Ham

#1 | Label: Ham | Cosine similarity: 0.33
Text: 0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I d

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer(max_features=20)
X = vectorizer.fit_transform(df['text'])
X_dense = X.toarray()

# Додаємо вектори окремо
lsh = LSH(num_buckets=10, num_hash_functions=5, input_dim=X_dense.shape[1])
for vec in X_dense[:500]:
    lsh.add_vector(vec)

# Робимо запит
#query_vec = X_dense[0]  # перший email
query_text = ["Do you want a free phone?"]
# Перетворюємо у TF-IDF вектор
query_vec = vectorizer.transform(query_text).toarray()[0]
similar_items = lsh.query(query_vec)

print('Схожих текстів: ', len(similar_items))
similar_items_sorted = sorted(
    similar_items,
    key=lambda x: cosine_similarity(query_vec.reshape(1, -1), x[1].reshape(1, -1))[0][0],
    reverse=True  # від більшого до меншого
)


Схожих текстів:  309


In [11]:
for idx, (i, vec) in enumerate(similar_items_sorted[:10]):
    similarity = np.dot(query_vec, vec) / (np.linalg.norm(query_vec) * np.linalg.norm(vec))
    print(f"#{idx+1} | Label: {'Spam' if df['label'].iloc[i]==1 else 'Ham'} | Cosine similarity: {similarity:.2f}")
    print(f"Text: {df['text']}...")

#1 | Label: Ham | Cosine similarity: 0.91
Text: 0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: text, Length: 5572, dtype: object...
#2 | Label: Ham | Cosine similarity: 0.83
Text: 0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already the